In [1]:
# ! pip install xgboost
# !pip install imblearn
# !pip install imbalanced-learn

In [2]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import confusion_matrix
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from imblearn.over_sampling  import RandomOverSampler
from sklearn.feature_extraction.text import TfidfTransformer


from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV

In [3]:
news_ids = pd.read_csv(r'..\data\news_outlets-accounts.csv')
news_ids.shape

(9636, 3)

In [4]:
news_ids.head()

,Token,Uid,Link
0,LouisvilleMag,11263532,https://twitter.com/LouisvilleMag
1,WindyCityTimes,30284449,https://twitter.com/WindyCityTimes
2,BlkMtnNews,3053471913,https://twitter.com/BlkMtnNews
3,NewYorker,14677919,https://twitter.com/NewYorker
4,KyKernel,41463945,https://twitter.com/KyKernel


In [5]:
input_path = r'..\data\preprocess_data.csv'
preprocess_data = (
            pd.read_csv(input_path, sep = "\t")
            .drop_duplicates()
        )
preprocess_data["Full Text"] =preprocess_data["Full Text"].fillna("")
preprocess_data.shape

(7125059, 21)

In [6]:
preprocess_data.head()

,Unnamed: 0,Date,Page Type,Account Type,Author,Full Name,Full Text,Gender,Hashtags,Impact,...,Thread Entry Type,Thread Author,Twitter Followers,Twitter Following,Twitter Tweets,Twitter Reply Count,Twitter Verified,Twitter Retweets,Reach (new),Region
0,0,2017-12-06 23:59:59,twitter,NaN,William82861606,William82861606 (William),funder conyers resigned franken resigning tomo...,male,"#trumpsexprobe, #metoo",0.0,...,share,funder,282.0,415.0,17340.0,0.0,False,0.0,0.0,Arizona
1,1,2017-12-06 23:59:58,twitter,NaN,Artis_Elemental,Artis_Elemental (ART...Is),alyssa_milano list sexual misconduct allegatio...,male,#metoo,24.5,...,share,Alyssa_Milano,5578.0,5769.0,363570.0,0.0,False,0.0,3311.0,NaN
2,2,2017-12-06 23:59:57,twitter,individual,lanah03,lanah03 (Laura Hopkins),deepinthehills sobbing silent longer,female,#metoo,5.4,...,share,DeepInTheHills,2713.0,4978.0,46914.0,0.0,False,0.0,1734.0,New York
3,3,2017-12-06 23:59:55,twitter,individual,lidskawasaki,lidskawasaki (dearlordbabyjesus),alyssa_milano time asking franken resign senat...,unknown,"#timepersonoftheyear, #metoo",0.0,...,share,Alyssa_Milano,330.0,273.0,10901.0,0.0,False,0.0,0.0,NaN
4,4,2017-12-06 23:59:54,twitter,NaN,IAmMeliLopez,IAmMeliLopez (Melissa Lopez),nycchr public hearing begun stories deserve he...,female,"#sexualharassment, #metoo",0.0,...,share,NYCCHR,13.0,96.0,11594.0,0.0,False,0.0,0.0,New York


In [7]:
preprocess_data.columns

Index(['Unnamed: 0', 'Date', 'Page Type', 'Account Type', 'Author',
       'Full Name', 'Full Text', 'Gender', 'Hashtags', 'Impact', 'Impressions',
       'Thread Entry Type', 'Thread Author', 'Twitter Followers',
       'Twitter Following', 'Twitter Tweets', 'Twitter Reply Count',
       'Twitter Verified', 'Twitter Retweets', 'Reach (new)', 'Region'],
      dtype='object')

In [ ]:
news_id_list = news_ids['Token'].tolist()
preprocess_data['news_authors'] = preprocess_data['Author'].apply(lambda x: x in news_id_list)

preprocess_data[['Author','news_authors']].head()

In [ ]:
preprocess_data[preprocess_data['news_authors']][['Author','news_authors','Full Text']].head()

In [ ]:
preprocess_data[preprocess_data['news_authors']][['Author','news_authors']].shape

In [ ]:
def news_identification_heuristic(author,tweet):
    # print(author,tweet)
    auth_heuristics = 'news' in str(author).lower() or 'times' in str(author).lower()
    tweet_heuristics = 'breaking news' in str(tweet).lower()
    return auth_heuristics or tweet_heuristics

In [ ]:
preprocess_data['news_authors_simp_heu'] = preprocess_data[['Author','Full Text']].\
    apply(lambda x: news_identification_heuristic(x[0],x[1]),axis=1)
# preprocess_data[['Author','news_authors_simp_heu']].head()
preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu','Full Text']].head()

In [ ]:
preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu']].shape


In [ ]:
preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu','news_authors','Full Text']].head(5)

In [ ]:
# news_id_list = [x.lower() for x in news_id_list]
def news_iden_imp_heu(author):
    # print(author,tweet)
    auth_heuristics = 'news' in str(author).lower() or author in news_id_list
    return 1 if auth_heuristics else 0

In [ ]:
preprocess_data["Full Text"] =preprocess_data["Full Text"].fillna("")
author_corpus = preprocess_data.groupby("Author")["Full Text"].apply(" ".join).reset_index()
author_corpus ['news_label'] = author_corpus['Author'].apply(news_iden_imp_heu)
author_corpus[author_corpus['news_label']==1]

In [ ]:
author_corpus

In [ ]:
vec = TfidfVectorizer(max_features = 10000)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
        author_corpus['Full Text'], 
        author_corpus['news_label'],
        random_state = 123,
        train_size = 0.75
    )

In [ ]:
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

In [ ]:
model = MultinomialNB()
model.fit(X_train_vec, y_train)

In [ ]:
train_preds = model.predict(X_train_vec)
test_preds = model.predict(X_test_vec)
train_preds

In [ ]:
print(classification_report(y_test, test_preds))
print(confusion_matrix(y_test, test_preds))

In [ ]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vec, y_train)

In [ ]:
train_preds = lr_tfidf.predict(X_train_vec)
test_preds = lr_tfidf.predict(X_test_vec)
# train_preds

In [ ]:
print(classification_report(y_test, test_preds))
print(confusion_matrix(y_test, test_preds))

In [ ]:
cv = CountVectorizer(max_features=10000, encoding="utf-8",  
      ngram_range = (1,3),  
      token_pattern = "[A-Za-z_][A-Za-z\d_]*")
X_train_vec = cv.fit_transform(X_train)
X_test_vec = cv.transform(X_test)

In [ ]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

# # make predictions for test data

# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]

In [ ]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters = {
    'grow_policy' : [0,1],
    'max_depth': range (2, 20, 1),
    'n_estimators': range(50, 250, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'balanced_accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

# xgb_model = grid_search.fit(X_train_vec, y_train)

In [ ]:
# xgb_model.param_grid

In [ ]:
nb_pipe2  = Pipeline([('vect',    CountVectorizer()),
                     ('tfidf',   TfidfTransformer()),
                     ('sampler', RandomOverSampler('minority',random_state=42)),
                     ('model',   MultinomialNB())])

In [ ]:
model = nb_pipe2.fit(X_train, y_train)

pred  = model.predict(X_test)

print(classification_report(pred,y_test))
print(confusion_matrix(pred,y_test))